In [14]:
import pandas as pd
import os
import joblib

In [15]:
stock_basic = pd.read_csv("stock_basic.csv")
stock_basic

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,1,平安银行,深圳,银行,19910403
1,000002.SZ,2,万科A,深圳,全国地产,19910129
2,000004.SZ,4,国华网安,深圳,软件服务,19910114
3,000006.SZ,6,深振业A,深圳,区域地产,19920427
4,000007.SZ,7,全新好,深圳,其他商业,19920413
...,...,...,...,...,...,...
4530,605580.SH,605580,恒盛能源,浙江,供气供热,20210819
4531,605588.SH,605588,冠石科技,江苏,元器件,20210812
4532,605589.SH,605589,圣泉集团,山东,化工原料,20210810
4533,605598.SH,605598,上海港湾,上海,建筑工程,20210917


In [16]:
trade_cal = pd.read_csv("trade_cal.csv")
trade_cal

,exchange,cal_date,is_open,pretrade_date
0,SSE,20141027,1,20141024
1,SSE,20141028,1,20141027
2,SSE,20141029,1,20141028
3,SSE,20141030,1,20141029
4,SSE,20141031,1,20141030
...,...,...,...,...
2427,SSE,20241021,1,20241018
2428,SSE,20241022,1,20241021
2429,SSE,20241023,1,20241022
2430,SSE,20241024,1,20241023


In [21]:
# factor_name = "return_-5"
# factor_name = "total_mv"
# factor_name = "mmt_intraday_M"
# factor_name = "vol_w_downshadow_std_1M"
# factor_name = "liq_amihud_avg_1M"
factor_name = "corr_ret_turn_post_1M"

In [22]:
 @joblib.delayed
def task(stock_name):
    current_df=pd.read_csv("factor/"+factor_name+"/"+stock_name+".csv")
    current_df=current_df[["factor"]]
    current_df = current_df.rename(columns={'factor': stock_name})
    return current_df

tasks=[]

for index, row in stock_basic.iterrows():
    stock_name = row.ts_code
#     print(stock_name)
    tasks.append(task(stock_name))

In [23]:
df = pd.concat(
    joblib.Parallel(n_jobs=5, verbose=10)(
        tasks
    ), axis=1)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    1.3s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    1.4s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    1.5s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    1.5s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1789s.) Setting batch_size=2.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    1.5s
[Parallel(n_jobs=5)]: Batch computation too fast (0.0778s.) Setting batch_size=4.
[Parallel(n_jobs=5)]: Done  45 tasks      | elapsed:    1.6s
[Parallel(n_jobs=5)]: Batch computation too fast (0.1254s.) Setting batch_size=8.
[Parallel(n_jobs=5)]: Done  79 tasks      | elapsed:    1.8s
[Parallel(n_jobs=5)]: Done 151 tasks      | elapsed:    2.2s
[Parallel(n_jobs=5)]: Done 255 tasks      | elapsed:    2.7s
[Parallel(n_jobs=5)]: Done 359 tasks      | elapsed:    3.2s
[Parallel(n_jobs=5)]: Done 479 tasks      | elapsed:    3.7s
[Paralle

In [24]:
df.to_csv(factor_name+".csv",index=0)

In [7]:
# df.index = trade_cal.cal_date.tolist()
# df

In [8]:
# df.tail(30)